# Algoritmos de optimización - Seminario - 

Nombre y Apellidos: Victor Joel Pinargote Bravo

Url GitHub: https://github.com/kmotillo/03MAIR_ALGORITMOS_OPTIMIZACION/upload/main/SEMINARIO/Trabajo_Final_Partidos_Liga.ipynb

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1qbG16dOSmKOXrQohQkfQGIJXTmpza-mN?usp=sharing)


# Problema 

> 2. Organizar los horarios de partidos de La Liga


## Descripción del problema

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que `maximice la audiencia`. 

Los horarios disponibles se conocen a priori y son los siguientes:


| Día | Horarios |
| -- | -- |
| Viernes | 20  |
| Sábado  | 12,16,18,20  |
| Domingo  | 12,16,18,20 |
| Lunes  |20 |

En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores (que tienen relación directa con la audiencia). Hay **3 equipos** en la **categoría A**, **11 equipos** de **categoría B** y **6 equipos** de **categoría C**. 


Se conoce estadisticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

|  | Categoria A | Categoria B | Categoria C |
|-- | -- | -- | -- |
| Categoria A | 2 Millones | 1,3 Millones | 1 Millones |
| Categoria B |  | 0,9 Millones | 0.75 Millones |
| Categoria C |  |  | 0.47 Millones |


Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla 

Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes Viernes Sábado Domingo Lunes

| | Viernes | Sábado | Domingo | Lunes |
| -- | -- | -- | -- | -- |
| 12h | - | 0.55 | 0.45 | - |
| 16h | - | 0.7 | 0.75 | - |
| 18h | - | 0.8 | 0.85 | - |
| 20h | 0.4 | 1 | 1 | 0.4 |

Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la siguiente tabla dependiendo del número de coincidencias:

| Coincidencias | -% |
| -- | -- |
| 0 | 0% |
| 1 | 25% |
| 2 | 45% |
| 3 | 60% |
| 4 | 70% |
| 5 | 75% |
| 6 | 78% |
| 7 | 80% |
| 8 | 80% |



## (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?

Tenemos 20 equipos con lo cual se celebran 10 encuentros y 10 Horarios disponibles si no existe restricción alguna estamos frente a una `variación con repeticiones` por lo que tendríamos $10^{10}$ posibilidades.

##  ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.

Si aplicamos todas las restricciones estamos frente a una `permutación sin repeticiones` con lo que tendríamos $10!$ de posibilidades.

## (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumenta la respuesta.

La estructura para los valores fijos o constanes que requieran de una identificacion usamos diccionarios que nos permitiran obtener los valores en función de las claves.

Para los valores de reducción de coincidencias usaremos un array ordenado para usarlo por el indice a modo de constantes.

Los valores del problema serán un dataframe generado a partir de los valores auxiliares (lista de equipos) y los datos necesarios para la ejecución del problema.


In [1]:
# Valores ordenados de los porcentajes de reduccion por coicidencia
PERCENTAGE_REDUCTION=[0,0.25,0.45,0.6,0.7,0.75,0.78,0.8,0,8]

#Coeficiente por horario
COEF_BY_TIMETABLE = [{'V20':0.4},{'S12':0.55},{'S16':0.7},{'S18':0.8},{'S20':1},{'D12':0.45},{'D16':0.75},{'D18':0.85},{'D20':1},{'L20':0.4}]

#Audiencias por categorias
AUDIENCE_BY_CATEGORY = {'AA':2e6,'AB':1.3e6,'AC':1e6,'BB':0.9e6,'BC':0.75e6,'CC':0.47e6}

#Equipos por categoria
TEAMS_BY_CATEGORY = {
    'A':['Real Madrid','R.Sociedad','Barcelona'],
    'B':['Celta','Valencia','Athletic','Villareal','Alavés','Levante','Espanyol','Sevilla','Betis','Atlético','Getafe'],
    'C':['Mallorca','Eibar','Leganés','Osasuna','Granada','Valladolid']
}

BASE_TEAMS = [{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['B'][0],TEAMS_BY_CATEGORY['A'][0]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['AB'],
    'category':'AB'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['B'][1],TEAMS_BY_CATEGORY['A'][1]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['AB'],
     'category':'AB'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['C'][0],TEAMS_BY_CATEGORY['C'][1]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['CC'],
    'category':'CC'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['B'][2],TEAMS_BY_CATEGORY['A'][2]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['AB'],
    'category':'AB'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['C'][2],TEAMS_BY_CATEGORY['C'][3]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['CC'],
    'category':'CC'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['B'][3],TEAMS_BY_CATEGORY['C'][4]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['BC'],
    'category':'BC'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['B'][4],TEAMS_BY_CATEGORY['B'][5]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['BB'],
    'category':'BB'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['B'][6],TEAMS_BY_CATEGORY['B'][7]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['BB'],
    'category':'BB'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['B'][8],TEAMS_BY_CATEGORY['C'][5]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['BC'],
    'category':'BC'
},
{
    'match':'{} vs {}'.format(TEAMS_BY_CATEGORY['B'][9],TEAMS_BY_CATEGORY['B'][10]),    
    'baseAudience':AUDIENCE_BY_CATEGORY['BB'],
    'category':'BB'
}
]
    
import pandas as pd

df_base = pd.DataFrame(BASE_TEAMS)
df_base

from functools import wraps
from time import time
def duration(f):
    @wraps(f)  
    def timing(*args, **kwargs):
        init = time()
        fun_result = f(*args, **kwargs)
        end = time()
        print('Elapsed time %d ms' %((end - init)*1000))
        return fun_result
    return timing



## (*)¿Cual es la función objetivo?

La función que buscamos es la sumatoria de los espectadores de la jornada aplicando las restricciones:
$$max(\sum_{i=1}^{10}x_i=a_i*t_i*(1-c_i))$$ 

Donde:
* `a` Es la audiencia del partido por categoria
* `t` Es el horario del partido
* `c` Es el coeficiente de reducción si existieran coincidencias

## (*) ¿Es un problema de maximización o minimización?

Como observamos en la función objetivo buscamos la suma de las audiencias de los partidos, con lo que para que el problema sea optimo debe máximizar la solución.

## Diseña un algoritmo para resolver el problema por fuerza bruta

In [2]:
from itertools import permutations

#Funcion que implementa la formula, en nuestros algoritmos no hay posibilidad de repetición de horarios con lo que no realizamos el ajuste por coincidencia
def calc_audience(row):  
  return row['baseAudience'] * list(row['timeTable'].values())[0]

@duration
def brute_force(df):  
  best_audience = 0
  best_solution = pd.DataFrame()
  #Obtenemos las 10! posibilidades de los horarios para comprobar uno a uno cual es el mejor  
  timeTablePermuted = list(permutations(COEF_BY_TIMETABLE))
  # Descomentar para ejecución completa, tarda alrededor de 3 horas en terminar , para el ejercicio usaremos 100000 de las permutaciones
  #for i in range(len(timeTablePermuted)):
  for i in range(100000):
    timeTableIteration = timeTablePermuted[i]
    new_audience = 0
    #Fusionamos el array de horarios al df              
    df['timeTable'] = timeTableIteration
    #Calculamos la audiencia en cada fila
    df['audience'] = df.apply(lambda row : calc_audience(row),1)
    #Realizamos el sumatorio de la audiencia
    new_audience = df['audience'].sum()
    #Comprobamos cual es la máxima audiencia y nos quedamos con ella
    if new_audience > best_audience:
      best_audience = new_audience
      best_solution = df.copy()    
  return best_solution,best_audience  
    

best_solution,best_audience = brute_force(df_base)
print("Best audience ",best_audience)
print("Best time table ")
best_solution
  
  

Elapsed time 132743 ms
Best audience  6537000.0
Best time table 


,match,baseAudience,category,timeTable,audience
0,Celta vs Real Madrid,1300000.0,AB,{'V20': 0.4},520000.0
1,Valencia vs R.Sociedad,1300000.0,AB,{'S18': 0.8},1040000.0
2,Mallorca vs Eibar,470000.0,CC,{'D12': 0.45},211500.0
3,Athletic vs Barcelona,1300000.0,AB,{'S20': 1},1300000.0
4,Leganés vs Osasuna,470000.0,CC,{'L20': 0.4},188000.0
5,Villareal vs Granada,750000.0,BC,{'S12': 0.55},412500.0
6,Alavés vs Levante,900000.0,BB,{'D16': 0.75},675000.0
7,Espanyol vs Sevilla,900000.0,BB,{'D18': 0.85},765000.0
8,Betis vs Valladolid,750000.0,BC,{'S16': 0.7},525000.0
9,Atlético vs Getafe,900000.0,BB,{'D20': 1},900000.0


## Calcula la complejidad del algoritmo por fuerza bruta

El algoritmo de fuerza bruta tiene una complejidad $O(n^2)$ ya que realizamos una iteración sobre las permutaciones de los horarios y realizamos una iteración, si bien enmascarada por `pandas` para el cálculo de la audiencia por fila del dataframe.

Este algoritmo es eficaz en su propósito, pero poco optimizado, ya que el número de iteraciones que se realiza es de $10!$ ($3.628.800$)

## (*)Diseña un algoritmo que mejore la complejidad del algoritmo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

El algoritmo que se presenta es bastante simple, nos apoyamos en la ordenación de los datos de los horarios y de los partidos por categoria y simplemente unimos estos dos resultados, dando el mejor resultado posible y con una implementación básica.


In [6]:
import random

#Función para seleccionar los equipos con una distribución aleatoria
def selectTeam(counters):
  #Mientras que no se seleccione un equipo iteramos
  while True:
    selected_team = random.choice(['A','B','C'])
    #Si hay equipos sin seleccionar por categoria lo obtenemos
    if (counters[selected_team]>0):
      team_name = TEAMS_BY_CATEGORY[selected_team][counters[selected_team]-1]
      #Retrocedemos en el contador para indicar que ya se han seleccionado todos los equipos de una categoria
      counters[selected_team] = counters[selected_team] - 1
      return selected_team,team_name

#Funcion para crear los partidos de manera aleatoria
def create_matchs():
  # Primero definimos la cantidad de equipos por categoria para controlar que se reparten bien
  teams_a = len(TEAMS_BY_CATEGORY['A'])
  teams_b = len(TEAMS_BY_CATEGORY['B'])
  teams_c = len(TEAMS_BY_CATEGORY['C'])
  #Asignamos los equipos a un diccionario para tener accesibles los contadores por categoria de equipo
  counters = {'A':teams_a,'B':teams_b,'C':teams_c}
  #Generamos un dataframe vacio con las columnas básicas
  matchs = pd.DataFrame(columns=['match','baseAudience','category'])
  #Generamos los 10 partidos de la jornada
  for i in range(10):
    #Seleccionamos el equipo local y visitante    
    selected_local = selectTeam(counters)
    selected_visit = selectTeam(counters)        
    categories = [selected_local[0],selected_visit[0]]
    #Ordenamos las categorias para cumplir con las keys definidas en el diccionario de audiencia
    categories.sort()
    match_category = "".join(categories)
    matchs = matchs.append({"match":"{} vs {}".format(selected_local[1],selected_visit[1]),"baseAudience":AUDIENCE_BY_CATEGORY[match_category],'category':match_category},ignore_index=True)  
  return matchs
  
@duration
def create_time_table_sorting(df):  
  #Ordenamos por coficiente los horarios
  time_table_sorted_by_coef = sorted(COEF_BY_TIMETABLE, key=lambda x: list(x.values())[0],reverse=True)  
  #Ordenamos los partidos por su categoria
  matchs_sorted = df.sort_values(['category']) 
  #Fusionamos el array de horarios ordenados al df ordenado
  matchs_sorted['timeTable'] = time_table_sorted_by_coef
  #Calculamos la audiencia en cada fila
  matchs_sorted['audience'] = matchs_sorted.apply(lambda row : calc_audience(row),1)  
  #Realizamos el sumatorio de la audiencia
  max_audience = matchs_sorted['audience'].sum()
  return matchs_sorted, max_audience

best_solution,best_audience = create_time_table_sorting(create_matchs())
print("Best audience ",best_audience)
print("Best time table ")
best_solution


Elapsed time 3 ms
Best audience  6713000.0
Best time table 


,match,baseAudience,category,timeTable,audience
3,R.Sociedad vs Sevilla,1300000.0,AB,{'S20': 1},1300000.0
5,Real Madrid vs Espanyol,1300000.0,AB,{'D20': 1},1300000.0
2,Granada vs Barcelona,1000000.0,AC,{'D18': 0.85},850000.0
1,Atlético vs Betis,900000.0,BB,{'S18': 0.8},720000.0
8,Villareal vs Athletic,900000.0,BB,{'D16': 0.75},675000.0
9,Valencia vs Celta,900000.0,BB,{'S16': 0.7},630000.0
0,Getafe vs Valladolid,750000.0,BC,{'S12': 0.55},412500.0
6,Eibar vs Levante,750000.0,BC,{'D12': 0.45},337500.0
7,Alavés vs Mallorca,750000.0,BC,{'V20': 0.4},300000.0
4,Osasuna vs Leganés,470000.0,CC,{'L20': 0.4},188000.0


## (*)Calcula la complejidad del algoritmo

Este algoritmo se basa en el algoritmo `sorted` que implementa un `Quick Sort` por defecto en python cuya complejidad es $O(n*log(n))$ , ya que la complejidad del cálculo de la audiencia es $O(n)$ nos quedamos como mayor complejidad la del algoritmo `sorted`

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorio.

Las funciones `create_matchs` y `selectTeam` se encargan de generar los partidos de manera aleatoria y coherente con los equipos y sus categorías.

## Describe brevemente en unas líneas como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño.

Creo que si aumnetamos la complejidad del problema (aumentar mas equipos) los algoritmos de poda serian una buena opción para resolverlos, generando la mayor audiencia posible por nodo y descartando los demás nodos. Por otro lado si aumentamos mucho mas la compejidad se podria abordar con algoritmos géneticos, la función de fitness y las mutaciones están bastante claras dentro del problema.